In [16]:

!pip install -q tfds-nightly




You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


In [117]:
import tensorflow_datasets as tfds
import tensorflow as tf

import time
import numpy as np
import matplotlib.pyplot as plt

In [118]:

examples, metadata = tfds.load('ted_hrlr_translate/pt_to_en', with_info=True,
                               as_supervised=True)
train_examples, val_examples = examples['train'], examples['validation']



In [119]:
one_pt,one_en=next(iter(train_examples))

In [76]:
# 这个类库不知道为啥我用不了，我换成tokenizer了
# 这个的功能除了分词，当遇到oov的时候相当于把词做切分了，比如Transformer就可能变成Trans和fromer两个词
# The tokenizer encodes the string by breaking it into subwords if the word is not in its dictionary.
tokenizer_en = tfds.deprecated.text.SubwordTextEncoder.build_from_corpus(
    (en.numpy() for pt, en in train_examples), target_vocab_size=2**13)

tokenizer_pt = tfds.deprecated.text.SubwordTextEncoder.build_from_corpus(
    (pt.numpy() for pt, en in train_examples), target_vocab_size=2**13)

AttributeError: module 'tensorflow_datasets' has no attribute 'deprecated'

In [120]:
# 只保留2**13个词 八千多
tokenizer_en=tf.keras.preprocessing.text.Tokenizer(num_words=2**13,oov_token="<UNK>")
tokenizer_en.fit_on_texts([en.numpy().decode('utf-8') for pt, en in train_examples])
tokenizer_pt=tf.keras.preprocessing.text.Tokenizer(num_words=2**13,oov_token="<UNK>")
tokenizer_pt.fit_on_texts([pt.numpy().decode('utf-8') for pt, en in train_examples])

In [121]:
sample_string = 'Transformer is awesome.'

tokenized_string = tokenizer_en.texts_to_sequences([sample_string])
print ('Tokenized string is {}'.format(tokenized_string))

original_string = tokenizer_en.sequences_to_texts(tokenized_string)
print ('The original string: {}'.format(original_string))


Tokenized string is [[1, 13, 1694]]
The original string: ['<UNK> is awesome']


In [122]:
BUFFER_SIZE = 20000
BATCH_SIZE = 64

def encode(lang1, lang2):
  # 与官网不同，我做了修改，使用的是tokenizer
  lang1 = [2**13] + tokenizer_pt.texts_to_sequences(
      [lang1.numpy().decode("utf8")])[0] + [2**13+1]

  lang2 = [2**13] + tokenizer_en.texts_to_sequences(
      [lang2.numpy().decode("utf8")])[0] + [2**13+1]

  return lang1, lang2

def tf_encode(pt, en):
  result_pt, result_en = tf.py_function(encode, [pt, en], [tf.int64, tf.int64])
  result_pt.set_shape([None])
  result_en.set_shape([None])

  return result_pt, result_en

MAX_LENGTH = 40
def filter_max_length(x, y, max_length=MAX_LENGTH):
  return tf.logical_and(tf.size(x) <= max_length,
                        tf.size(y) <= max_length)

encode(one_pt,one_en)

([8192, 5, 35, 1, 2, 761, 3901, 2, 358, 2257, 18, 1702, 4, 8, 2, 1, 8193],
 [8192,
  3,
  53,
  12,
  1085,
  1,
  12,
  86,
  117,
  237,
  2,
  39,
  2009,
  5,
  2286,
  84,
  13,
  1,
  8193])

In [123]:
train_dataset = train_examples.map(tf_encode)
train_dataset = train_dataset.filter(filter_max_length)
# cache the dataset to memory to get a speedup while reading from it.
train_dataset = train_dataset.cache()
train_dataset = train_dataset.shuffle(BUFFER_SIZE).padded_batch(BATCH_SIZE)
train_dataset = train_dataset.prefetch(tf.data.experimental.AUTOTUNE)


val_dataset = val_examples.map(tf_encode)
val_dataset = val_dataset.filter(filter_max_length).padded_batch(BATCH_SIZE)


In [124]:
pt_batch, en_batch = next(iter(val_dataset))
pt_batch, en_batch

(<tf.Tensor: shape=(64, 39), dtype=int64, numpy=
 array([[8192,  282, 7964, ...,    0,    0,    0],
        [8192,   87,  136, ...,    0,    0,    0],
        [8192, 4239,    9, ...,    0,    0,    0],
        ...,
        [8192,   87,    2, ...,    0,    0,    0],
        [8192,   26, 7434, ...,    0,    0,    0],
        [8192,   21, 1247, ...,    0,    0,    0]])>,
 <tf.Tensor: shape=(64, 40), dtype=int64, numpy=
 array([[8192,   85,   18, ...,    0,    0,    0],
        [8192,    8,   19, ...,    0,    0,    0],
        [8192,    8, 2251, ...,    0,    0,    0],
        ...,
        [8192,    9,   19, ...,    0,    0,    0],
        [8192,    8, 1562, ...,    0,    0,    0],
        [8192,  941,   10, ...,    0,    0,    0]])>)

In [125]:
tokenizer_pt.word_docs

defaultdict(int,
            {'e': 17266,
             'procura': 105,
             'tiramos': 15,
             'é': 12290,
             'a': 20017,
             'melhoramos': 2,
             'vantagem': 31,
             'que': 19106,
             'única': 229,
             'serendipidade': 1,
             'quando': 2285,
             'impressão': 40,
             'da': 5169,
             'ativos': 17,
             'estes': 711,
             'fossem': 65,
             'mas': 4967,
             'se': 6894,
             'fatores': 29,
             'não': 7902,
             'me': 2390,
             'tinham': 281,
             'eles': 1413,
             'de': 18808,
             'testar': 44,
             'curiosidade': 32,
             'como': 4715,
             'esta': 1904,
             'razão': 265,
             'agnóstica': 1,
             'ainda': 801,
             'ter': 1195,
             'consciente': 23,
             'rebeldia': 2,
             'posso': 264,
             'qual': 

In [126]:
"""
终于通过代码看懂了Positional encoding是个啥
相当于对位置信息进行embedding，假如说有一个长度为50的序列，那么对这个序列的每个位置进行编码，假如需要的维度为512
对于其第pos个位置（例如第0个位置，pos就是0），先计算一个pos/10000^{2*i/512}的向量，i从0一直到511，所以这是一个512维度的向量
然后如果这个pos是一个奇数，就对其计算cos，否则计算sin
最终的position encoding就是一个50*512的向量
"""

def get_angles(pos, i, d_model):
  print(pos.shape)
  print(i.shape)
  angle_rates = 1 / np.power(10000, (2 * (i//2)) / np.float32(d_model))
  print(angle_rates.shape)
  return pos * angle_rates

def positional_encoding(position, d_model):
  angle_rads = get_angles(np.arange(position)[:, np.newaxis],
                          np.arange(d_model)[np.newaxis, :],
                          d_model)
  print(angle_rads.shape)
  # apply sin to even indices in the array; 2i
  angle_rads[:, 0::2] = np.sin(angle_rads[:, 0::2])

  # apply cos to odd indices in the array; 2i+1
  angle_rads[:, 1::2] = np.cos(angle_rads[:, 1::2])
  pos_encoding = angle_rads[np.newaxis, ...]

  return tf.cast(pos_encoding, dtype=tf.float32)

pos_encoding = positional_encoding(50, 512)
print (pos_encoding)

(50, 1)
(1, 512)
(1, 512)
(50, 512)
tf.Tensor(
[[[ 0.          1.          0.         ...  1.          0.
    1.        ]
  [ 0.84147096  0.5403023   0.8218562  ...  1.          0.00010366
    1.        ]
  [ 0.9092974  -0.41614684  0.9364147  ...  1.          0.00020733
    1.        ]
  ...
  [ 0.12357312 -0.9923355   0.97718984 ...  0.99998724  0.00487216
    0.99998814]
  [-0.76825464 -0.64014435  0.7312359  ...  0.9999867   0.00497582
    0.9999876 ]
  [-0.95375264  0.30059254 -0.14402692 ...  0.9999861   0.00507948
    0.9999871 ]]], shape=(1, 50, 512), dtype=float32)


In [127]:
def create_padding_mask(seq):
  seq = tf.cast(tf.math.equal(seq, 0), tf.float32)

  # add extra dimensions to add the padding
  # to the attention logits.
  return seq[:, tf.newaxis, tf.newaxis, :]  # (batch_size, 1, 1, seq_len)

x = tf.constant([[7, 6, 0, 0, 1], [1, 2, 3, 0, 0], [0, 0, 0, 4, 5]])
create_padding_mask(x)

<tf.Tensor: shape=(3, 1, 1, 5), dtype=float32, numpy=
array([[[[0., 0., 1., 1., 0.]]],


       [[[0., 0., 0., 1., 1.]]],


       [[[1., 1., 1., 0., 0.]]]], dtype=float32)>

In [128]:
def create_look_ahead_mask(size):
  # 这是个有趣的api，相当于对一个矩阵斜着进行遮罩
  mask = 1 - tf.linalg.band_part(tf.ones((size, size)), -1, 0)
  return mask  # (seq_len, seq_len)

x = tf.random.uniform((1, 3))
temp = create_look_ahead_mask(x.shape[1])
temp

<tf.Tensor: shape=(3, 3), dtype=float32, numpy=
array([[0., 1., 1.],
       [0., 0., 1.],
       [0., 0., 0.]], dtype=float32)>

In [129]:
def scaled_dot_product_attention(q, k, v, mask):
  """Calculate the attention weights.
  q, k, v must have matching leading dimensions.
  k, v must have matching penultimate dimension, i.e.: seq_len_k = seq_len_v.
  The mask has different shapes depending on its type(padding or look ahead)
  but it must be broadcastable for addition.

  Args:
    q: query shape == (..., seq_len_q, depth)
    k: key shape == (..., seq_len_k, depth)
    v: value shape == (..., seq_len_v, depth_v)
    mask: Float tensor with shape broadcastable
          to (..., seq_len_q, seq_len_k). Defaults to None.
  这代码写得真好，看明白了
  Returns:
    output, attention_weights
  """
  # q和k的最后两维度被当做矩阵进行相乘
  matmul_qk = tf.matmul(q, k, transpose_b=True)  # (..., seq_len_q, seq_len_k)

  # scale matmul_qk
  dk = tf.cast(tf.shape(k)[-1], tf.float32)
  scaled_attention_logits = matmul_qk / tf.math.sqrt(dk)

  # add the mask to the scaled tensor.
  if mask is not None:
    scaled_attention_logits += (mask * -1e9)

  # softmax is normalized on the last axis (seq_len_k) so that the scores
  # add up to 1.
  attention_weights = tf.nn.softmax(scaled_attention_logits, axis=-1)  # (..., seq_len_q, seq_len_k)

  output = tf.matmul(attention_weights, v)  # (..., seq_len_q, depth_v)

  return output, attention_weights

In [130]:
def print_out(q, k, v):
  temp_out, temp_attn = scaled_dot_product_attention(
      q, k, v, None)
  print ('Attention weights are:')
  print (temp_attn)
  print ('Output is:')
  print (temp_out)
  
np.set_printoptions(suppress=True)

temp_k = tf.constant([[10,0,0],
                      [0,10,0],
                      [0,0,10],
                      [0,0,10]], dtype=tf.float32)  # (4, 3)

temp_v = tf.constant([[   1,0],
                      [  10,0],
                      [ 100,5],
                      [1000,6]], dtype=tf.float32)  # (4, 2)

# This `query` aligns with the second `key`,
# so the second `value` is returned.
temp_q = tf.constant([[0, 10, 0]], dtype=tf.float32)  # (1, 3)
print_out(temp_q, temp_k, temp_v)

print("牛逼")

# This query aligns equally with the first and second key,
# so their values get averaged.
temp_q = tf.constant([[10, 10, 0]], dtype=tf.float32)  # (1, 3)
print_out(temp_q, temp_k, temp_v)
print("牛逼")

# This query aligns with a repeated key (third and fourth),
# so all associated values get averaged.
temp_q = tf.constant([[0, 0, 10]], dtype=tf.float32)  # (1, 3)
print_out(temp_q, temp_k, temp_v)
print("牛逼")

Attention weights are:
tf.Tensor([[0. 1. 0. 0.]], shape=(1, 4), dtype=float32)
Output is:
tf.Tensor([[10.  0.]], shape=(1, 2), dtype=float32)
牛逼
Attention weights are:
tf.Tensor([[0.5 0.5 0.  0. ]], shape=(1, 4), dtype=float32)
Output is:
tf.Tensor([[5.5 0. ]], shape=(1, 2), dtype=float32)
牛逼
Attention weights are:
tf.Tensor([[0.  0.  0.5 0.5]], shape=(1, 4), dtype=float32)
Output is:
tf.Tensor([[550.    5.5]], shape=(1, 2), dtype=float32)
牛逼


In [ ]:
class MultiHeadAttention(tf.keras.layers.Layer):
  def __init__(self, d_model, num_heads):
    super(MultiHeadAttention, self).__init__()
    self.num_heads = num_heads
    self.d_model = d_model

    assert d_model % self.num_heads == 0

    self.depth = d_model // self.num_heads

    self.wq = tf.keras.layers.Dense(d_model)
    self.wk = tf.keras.layers.Dense(d_model)
    self.wv = tf.keras.layers.Dense(d_model)

    self.dense = tf.keras.layers.Dense(d_model)

  def split_heads(self, x, batch_size):
    """Split the last dimension into (num_heads, depth).
    Transpose the result such that the shape is (batch_size, num_heads, seq_len, depth)

    这个玩意的功能就比如说，输入的是(batch_size,seq_len,d_model)，然后把最后一个depth分割成(batch_size,num_heads,seq_len,depth)
    reshape的-1代表自动计算
    """
    print(x.shape)
    x = tf.reshape(x, (batch_size, -1, self.num_heads, self.depth))
    return tf.transpose(x, perm=[0, 2, 1, 3])

  def call(self, v, k, q, mask):
    batch_size = tf.shape(q)[0]

    q = self.wq(q)  # (batch_size, seq_len, d_model)
    k = self.wk(k)  # (batch_size, seq_len, d_model)
    v = self.wv(v)  # (batch_size, seq_len, d_model)

    q = self.split_heads(q, batch_size)  # (batch_size, num_heads, seq_len_q, depth)
    k = self.split_heads(k, batch_size)  # (batch_size, num_heads, seq_len_k, depth)
    v = self.split_heads(v, batch_size)  # (batch_size, num_heads, seq_len_v, depth)

    # scaled_attention.shape == (batch_size, num_heads, seq_len_q, depth)
    # attention_weights.shape == (batch_size, num_heads, seq_len_q, seq_len_k)
    scaled_attention, attention_weights = scaled_dot_product_attention(
        q, k, v, mask)

    scaled_attention = tf.transpose(scaled_attention, perm=[0, 2, 1, 3])  # (batch_size, seq_len_q, num_heads, depth)

    concat_attention = tf.reshape(scaled_attention,
                                  (batch_size, -1, self.d_model))  # (batch_size, seq_len_q, d_model)

    output = self.dense(concat_attention)  # (batch_size, seq_len_q, d_model)

    return output, attention_weights

tf.Tensor(
[[[ 0  1  2  3  4  5]
  [ 6  7  8  9 10 11]
  [12 13 14 15 16 17]]

 [[18 19 20 21 22 23]
  [24 25 26 27 28 29]
  [30 31 32 33 34 35]]], shape=(2, 3, 6), dtype=int32)
tf.Tensor(
[[[[ 0  1]
   [ 2  3]
   [ 4  5]]

  [[ 6  7]
   [ 8  9]
   [10 11]]

  [[12 13]
   [14 15]
   [16 17]]]


 [[[18 19]
   [20 21]
   [22 23]]

  [[24 25]
   [26 27]
   [28 29]]

  [[30 31]
   [32 33]
   [34 35]]]], shape=(2, 3, 3, 2), dtype=int32)
tf.Tensor(
[[[[ 0  1]
   [ 2  3]
   [ 4  5]]

  [[ 6  7]
   [ 8  9]
   [10 11]]

  [[12 13]
   [14 15]
   [16 17]]]


 [[[18 19]
   [20 21]
   [22 23]]

  [[24 25]
   [26 27]
   [28 29]]

  [[30 31]
   [32 33]
   [34 35]]]], shape=(2, 3, 3, 2), dtype=int32)
tf.Tensor(
[[[[ 0  1  2]
   [ 3  4  5]
   [ 6  7  8]]

  [[ 9 10 11]
   [12 13 14]
   [15 16 17]]]


 [[[18 19 20]
   [21 22 23]
   [24 25 26]]

  [[27 28 29]
   [30 31 32]
   [33 34 35]]]], shape=(2, 2, 3, 3), dtype=int32)
